In [2]:
import re
import pandas as pd
import nltk
from flask import Flask, request, jsonify
from nltk.corpus import stopwords
import torch
from sklearn.model_selection import train_test_split
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, GPT2Tokenizer, GPT2LMHeadModel

c:\Users\Suyash Tambe\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd
import json

# Load the JSON dataset
with open(r'C:\Users\Suyash Tambe\Desktop\shayari generator\quotes.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Convert JSON data to a pandas DataFrame
df_json = pd.DataFrame(json_data)


In [6]:
# Load the TXT dataset
with open(r'C:\Users\Suyash Tambe\Desktop\shayari generator\hindi_shayari.txt', 'r', encoding='utf-8') as file:
    txt_data = file.readlines()

# Convert TXT data to a pandas DataFrame
df_txt = pd.DataFrame(txt_data, columns=['text'])


In [ ]:
# Load the CSV datasets
f
df_csv = pd.read_csv(r'C:\Users\Suyash Tambe\Desktop\shayari generator\quotes_filtered_2.csv')


In [8]:
# Concatenate all DataFrames
combined_df = pd.concat([df_json, df_txt, df_csv], ignore_index=True)
# Save combined dataset
combined_df.to_csv('combined_quotes.csv', index=False, encoding='utf-8')
print(f'Total combined records: {len(combined_df)}')

Total combined records: 53655


In [2]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\Suyash
[nltk_data]     Tambe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def preprocess_text(text):
    if isinstance(text, str):
        # Remove special characters
        text = re.sub(r'\W', ' ', text)
        # Remove single characters
        text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
        # Remove multiple spaces
        text = re.sub(r'\s+', ' ', text, flags=re.I)
        # Convert to lowercase
        text = text.lower()
        # Remove stopwords
        text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [ ]:
# Convert all entries to strings and handle missing values
combined_df['text'] = combined_df['text'].astype(str).fillna('')

# Apply preprocessing to the text column
combined_df['processed_text'] = combined_df['text'].apply(preprocess_text)

# Save the preprocessed data to a CSV file
combined_df.to_csv('processed_shayaris_quotes.csv', index=False, encoding='utf-8')

print(f'Total combined records: {len(combined_df)}')

In [5]:
model_name = 'gpt2-medium'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [8]:

# Check if CUDA is available and set the device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load your pretrained GPT-2 model and tokenizer
model_name = 'gpt2-medium'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

# Load your combined dataset
df = pd.read_csv(r'C:\Users\Suyash Tambe\Desktop\shayari generator\quotes.csv')

# Split the dataset into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the splits to csv
train_df.to_csv('train_shayaris_quotes.csv', index=False)
test_df.to_csv('test_shayaris_quotes.csv', index=False)

# Function to load dataset for language model training
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset

# Load train and test datasets
train_dataset = load_dataset('train_shayaris_quotes.csv', tokenizer)
test_dataset = load_dataset('test_shayaris_quotes.csv', tokenizer)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,  # Adjust batch size if necessary
    save_steps=7000,
    save_total_limit=2,
    fp16=True if torch.cuda.is_available() else False,  # Enable mixed precision training if GPU is available
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Start training
trainer.train()

# Save the model and tokenizer
model.save_pretrained('./gpt_model')
tokenizer.save_pretrained('./gpt_model')

Using device: cuda


c:\Users\Suyash Tambe\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
  1%|          | 500/55809 [43:05<82:05:01,  5.34s/it]

{'loss': 3.4331, 'grad_norm': 4.322802543640137, 'learning_rate': 4.955473131573761e-05, 'epoch': 0.01}


  2%|▏         | 1000/55809 [1:26:27<77:27:36,  5.09s/it]

{'loss': 3.3456, 'grad_norm': 3.2921230792999268, 'learning_rate': 4.910677489293842e-05, 'epoch': 0.02}


...

 99%|█████████▉| 55500/55809 [76:55:44<27:26,  5.33s/it]  

{'loss': 2.9209, 'grad_norm': 1.9031325578689575, 'learning_rate': 2.974430647386622e-07, 'epoch': 0.99}


100%|██████████| 55809/55809 [77:22:49<00:00,  4.99s/it]


{'train_runtime': 278569.6543, 'train_samples_per_second': 0.801, 'train_steps_per_second': 0.2, 'train_loss': 3.0053791757762807, 'epoch': 1.0}


('./trained_model\\tokenizer_config.json',
 './trained_model\\special_tokens_map.json',
 './trained_model\\vocab.json',
 './trained_model\\merges.txt',
 './trained_model\\added_tokens.json')

In [9]:
model.save_pretrained('./gpt_model')
tokenizer.save_pretrained('./gpt_model')

('./gpt_model\\tokenizer_config.json',
 './gpt_model\\special_tokens_map.json',
 './gpt_model\\vocab.json',
 './gpt_model\\merges.txt',
 './gpt_model\\added_tokens.json')

In [3]:
model_path = r'C:\Users\Suyash Tambe\Desktop\shayari generator\gpt_model'
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

## Testing ##

In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

c:\Users\Suyash Tambe\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


model_name = r'C:\Users\Suyash Tambe\Desktop\shayari generator\gpt_model'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

def generate_quote(tags, max_length=100, temperature=1.0, top_k=50, top_p=0.95):
    # Encode the input tags
    input_ids = tokenizer.encode(tags, return_tensors='pt').to(device)
    
    # Generate the quote
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id,
        temperature=temperature,      
        top_k=top_k,                  
        top_p=top_p,                  
        do_sample=True,               
        eos_token_id=tokenizer.eos_token_id, # Ensure sentences end properly
    )
    
    # Decode the output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Split the text into sentences and take the first one
    sentences = generated_text.split('.')
    if len(sentences) > 1:
        generated_text = sentences[0] + '.'
    
    return generated_text




Using device: cuda


In [4]:
# Example usage
tags_list = ["relationship", "happiness, love"]
quote_counter = 1

for tags in tags_list:
    print(f"Generating quotes for tags: {tags}")
    for i in range(5):  # Generate multiple quotes for each set of tags
        quote = generate_quote(tags)
        print(f"{quote_counter}. {quote}")
        quote_counter += 1

Generating quotes for tags: relationship
1. relationship, romance"
"Love and kindness don’t come in packages, you can have them both, but you cannot get them all the time and still leave home feeling inferior.
2. relationship"
"In a few years, the world will have enough computers to take down entire nations, and we won’t even know who is responsible for that.
3. relationship, relationship-quotes, self-care, women-writers"
"I'm interested in the way the public, especially in Washington, feels that there are no real differences between a liberal and a conservative and between the two political parties; that the country is united, not divided.
4. relationship
"She had found a way to live in the past, a place in which she would never be the same; a moment in a lifetime she was gone.
5. relationship-advice, love, relationship-quotes"
"As a writer, a poet has a great deal to give.
Generating quotes for tags: happiness, love
6. happiness, love, pain, sorrow, suffer, suffering"
"I feel like I'